<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком.

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
# загрузим версию 1.1.3 библиотеки scikit-learn
# для последующей корректной работы с OneHotEncoder

!pip install scikit-learn==1.1.3

In [2]:
# Загрузим библиотеку imblearn для изменения размера выборок

!pip install imblearn

In [3]:
# Импортируем нужные функции и библиотеки

from collections import Counter

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score, recall_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.dummy import DummyClassifier

from tqdm import tqdm

In [4]:
# Сохраним в константу random_state

RANDOM_STATE = 12345

In [5]:
# Вынесем в отдельную функцию вывод размера выборки и первых 5 строк

def sample_info(data):
    display(data.head())
    display(data.shape)

In [6]:
# Далее по ходу проекта очень часто используется один и тот же код расчета метрик f1-меры и auc-roc,
# f1-мера используется при подборе гиперпараметров модели, а вот расчет auc-roc можно вычислять отдельной функцией

def calculation_of_aucroc(model, features, target):
    probabilities_valid = model.predict_proba(features)
    probabilities_one_valid = probabilities_valid[:, 1]
    auc_roc = roc_auc_score(target, probabilities_one_valid)
    return auc_roc

In [7]:
# Загружаем данные  из файла

try:
    data = pd.read_csv('Churn.csv')
except:
    data = pd.read_csv('/datasets/Churn.csv')

sample_info(data)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


(10000, 14)

In [8]:
len(data['Surname'].unique())

2932

In [9]:
len(data['Geography'].unique())

3

In [10]:
len(data['Gender'].unique())

2

In [11]:
# Проверим пропуски в данных

data.isna().sum()

RowNumber            0
CustomerId           0
Surname              0
CreditScore          0
Geography            0
Gender               0
Age                  0
Tenure             909
Balance              0
NumOfProducts        0
HasCrCard            0
IsActiveMember       0
EstimatedSalary      0
Exited               0
dtype: int64

**Выводы по данным**

1. Изучив данные, мы видим, что у нас есть три столбца с категориальными признаками: Фамилия, страна проживания и пол. Страну проживания и пол вполне логично обработать техникой ***OHE***.
2. Столбец с фамилией клиента лучше исключить из датафрейма, так как уникальных значений в столбце 2932. То есть к клиенту при необходимости доступ можно будет получить по его идентификатору, а при преобразовании техникой ***OHE*** у нас добавиться огромное количество бессмысленных столбцов.
3. Столбец индексом строки в данных также можно исключить, поскольку он дублирует индексы датафрейма.
4. Пропуски в столбце ~~заменим значением `-1`, что однозначно будет говорить о пропуске.~~ `Tenure` исключим из датафрейма.

In [12]:
# Ограничим данные, исключив столбцы с фамилией, номером строки и идентификатором клиента

data_without_surname = data.drop(['Surname', 'RowNumber', 'CustomerId'], axis=1)
sample_info(data_without_surname)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


(10000, 11)

In [13]:
# Удалим строки с пропусками в столбце Tenure

# data_without_surname = data_without_surname.dropna(subset=['Tenure'])
median = data_without_surname['Tenure'].median()
data_without_surname['Tenure'] = data_without_surname['Tenure'].fillna(median)
sample_info(data_without_surname)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


(10000, 11)

In [14]:
# Отделим целевой признак от признаков и разделим набор данных на выборки

target = data_without_surname['Exited']
features = data_without_surname.drop(['Exited'], axis=1)

features, features_test, target, target_test = train_test_split(features,
                                                                target, test_size=0.20,
                                                                random_state=RANDOM_STATE)
features_train, features_valid, target_train, target_valid = train_test_split(features, target,
                                                                              test_size=0.25,
                                                                              random_state=RANDOM_STATE)
print(features_test.shape)
print(features_train.shape)
print(features_valid.shape)

(2000, 10)
(6000, 10)
(2000, 10)


Далее проведем преобразование категориальных признаков тренировочной выборки техникой **OneHotEncoding** и масштабирование методом стандартизации

In [15]:
# Выделим столбцы с категориальными признаками для преобразования

ohe_columns = features_train.select_dtypes(include='object').columns.to_list()
ohe_columns

['Geography', 'Gender']

In [16]:
# вынесем в переменную numeric названия столбцов с численными признаками, они нам понадобятся при масштабировании

numeric = features_train.select_dtypes(exclude='object').columns.to_list()
numeric

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [17]:
# Создадим экземпляр класса OneHotEncoder

encoder_ohe = OneHotEncoder(drop='first', handle_unknown='ignore', sparse=False)

# Обучим энкодер на заданных категориальных признаках

encoder_ohe.fit(features_train[ohe_columns])

# Добавим закодированные признаки в тренировочную выборку

features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train[ohe_columns])

# Удалим из выборки незакодированные столбцы

features_train = features_train.drop(ohe_columns, axis=1)
sample_info(features_train)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
492,639,38,4.0,81550.94,2,0,1,118974.77,0.0,0.0,0.0
6655,554,44,5.0,85304.27,1,1,1,58076.52,0.0,0.0,1.0
4287,714,53,1.0,99141.86,1,1,1,72496.05,1.0,0.0,1.0
42,556,61,2.0,117419.35,1,1,1,94153.83,0.0,0.0,0.0
8178,707,46,7.0,127476.73,2,1,1,146011.55,0.0,0.0,0.0


(6000, 11)

In [18]:
# Теперь проведем масштабирование
# Создадим экземпляр класса StandardScaler

scaler = StandardScaler()

# Обучим его на тренировочных данных и трансформируем их

features_train[numeric] = scaler.fit_transform(features_train[numeric])

sample_info(features_train)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
492,-0.134048,-0.078068,-0.369113,0.076163,0.816929,-1.550255,0.968496,0.331571,0.0,0.0,0.0
6655,-1.010798,0.494555,-0.007415,0.136391,-0.896909,0.645055,0.968496,-0.727858,0.0,0.0,1.0
4287,0.639554,1.353490,-1.454209,0.358435,-0.896909,0.645055,0.968496,-0.477006,1.0,0.0,1.0
42,-0.990168,2.116987,-1.092511,0.651725,-0.896909,0.645055,0.968496,-0.100232,0.0,0.0,0.0
8178,0.567351,0.685430,0.715982,0.813110,0.816929,0.645055,0.968496,0.801922,0.0,0.0,0.0


(6000, 11)

Мы видим, что теперь в нашей таблице представлены только численные столбцы и можно работать с данными дальше, но перед этим мы масштабируем численные значения в столбцах методом стандартизации.

In [19]:
# теперь можно провести кодирование и масштабирование валидационной выборки

features_valid[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_valid[ohe_columns])
features_valid = features_valid.drop(ohe_columns, axis=1)

features_valid[numeric] = scaler.transform(features_valid[numeric])

# Посмотрим, что получилось

sample_info(features_valid)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
2358,0.175393,0.399118,-1.454209,1.385698,-0.896909,-1.550255,0.968496,-1.466761,0.0,0.0,1.0
8463,-1.299609,0.971741,-1.092511,-1.232442,-0.896909,0.645055,-1.032529,0.254415,0.0,1.0,1.0
163,0.711757,-0.268942,-1.092511,-1.232442,0.816929,0.645055,0.968496,0.122863,0.0,1.0,0.0
3074,-0.391916,0.494555,0.354284,0.672529,-0.896909,0.645055,-1.032529,0.585847,1.0,0.0,0.0
5989,0.165078,1.353490,1.801078,0.536522,-0.896909,-1.550255,-1.032529,1.462457,0.0,0.0,0.0


(2000, 11)

In [20]:
# И всё то же самое с тестовой выборкой

features_test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_test[ohe_columns])
features_test = features_test.drop(ohe_columns, axis=1)

features_test[numeric] = scaler.transform(features_test[numeric])

# Посмотрим, что получилось

sample_info(features_test)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
7867,-0.123733,0.685430,-0.730812,-1.232442,-0.896909,0.645055,0.968496,0.980212,0.0,1.0,0.0
1402,1.083087,-0.937002,1.077681,0.858518,-0.896909,0.645055,-1.032529,-0.390486,0.0,0.0,1.0
8606,1.598822,0.303681,-0.007415,-1.232442,0.816929,0.645055,0.968496,-0.435169,0.0,1.0,1.0
8885,0.165078,0.589993,-0.369113,0.412100,0.816929,0.645055,0.968496,1.017079,0.0,1.0,1.0
6494,0.484834,-1.032439,0.715982,-1.232442,0.816929,0.645055,0.968496,-1.343558,0.0,0.0,1.0


(2000, 11)

<div class='alert alert-info'>
<b>Корректировка студента</b><br><br>
В ходе подготовки данных к дальнейшей работе с моделями <b><i>обнаружилось:</i></b>
<ol>
<li>В таблице было три столбца, которые явно не несли какой-либо ценной информации для обучения моделей (номер строки, фамилия клиента, идентификатор клиента).</li>
<li>В таблице 3 столбца с категориальными признаками, которые не подходят для обучения моделей.</li>
<li>В столбце с количеством лет было 909 строк с пропусками.</li>
</ol>
<br><br>
В связи с чем были <b><i>проведены</i></b> следующие манипуляции:
<ol>
<li>Три лишних столбца были полностью исключены из датафрейма.</li>
<li>Строки с пропусками были исключены из датафрейма по той причине, что любая замена, пусть немного, но исказит информацию.</li>
<li>Оставшиеся два столбца с категориальными признаками были преобразованы техникой OHE (сначала на тренировочных данных, затем - на валидационных и тестовых).</li>
<li>Все данные в численных столбцах промасштабированы (также, сначала на тренировочной выборке, потом - на валидационной и тестовой).</li>
</ol>
</div>

## Исследование задачи

Для начала "взвесим" классы.

In [21]:
f'{Counter(target_train)}'

'Counter({0: 4781, 1: 1219})'

Мы видим, что класс "0" встречается почти в 4 раза чаще, чем класс "1". У нас в данных наблюдается дисбаланс.
Попробуем обучить модель без учета баланса классов и посмотрим, что из этого получится.

In [22]:
# Обучим модель логистической регрессии и посмотрим, что нам покажут метрики

model = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_disbalance = f1_score(target_valid, predicted_valid)

f'f1: {f1_disbalance}, auc-roc: {calculation_of_aucroc(model, features_valid, target_valid)}'

'f1: 0.30131826741996237, auc-roc: 0.7703010082353259'

Теперь поработаем с моделью "Дерево решений" на несбалансированных данных.

Результаты будем сразу сравнивать с результатами обученной логистической регрессии, если модель ни при какой глубине не покажет лучших метрик, то мы не станем ее даже рассматривать.

In [23]:
# Подберем наилучшие гиперпараметры

best_depth = 0
best_f1 = f1_disbalance
best_model = model
for depth in range(1, 20):
    model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predicted_valid = model_tree.predict(features_valid)
    f1_tree = f1_score(target_valid, predicted_valid)
    if f1_tree > best_f1:
        best_depth = depth
        best_f1 = f1_tree
        best_model = model_tree

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}'

'f1: 0.5583596214511041, auc-roc: 0.8231010349393358, depth: 7'

Итак, при глубине 9, модель **решающего дерева** показала неплохие результаты даже на несбалансированных данных.

Теперь поработаем с моделью **случайного леса** и будем сразу сравнивать результаты с полученными на предыдущем этапе.

In [24]:
best_depth = 0
best_est = 0
best_f1 = best_f1
best_model = best_model

for depth in range(1, 20):
    for est in range(10, 170, 10):
        model = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=depth, n_estimators=est)
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_depth = depth
            best_est = est
            best_f1 = f1
            best_model = model

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}, est: {best_est}'

'f1: 0.5805422647527911, auc-roc: 0.8379050704238784, depth: 14, est: 40'

In [25]:
%%time
# RandomForestClassifier дал лучшие метрики качества на несбалансированных данных,
# проверим сколько времени модель будет обучаться с полученными гиперпараметрами

best_model.fit(features_train, target_train)

Wall time: 212 ms


RandomForestClassifier(max_depth=14, n_estimators=40, random_state=12345)

~~Метрика **AUC-ROC** показывает неплохие результаты, а вот **F1-меру** хотелось бы получше.~~

На несбалансированных данных лучшие результаты показала модель **RandomForestClassifier** с гиперпараметрами **max_depth=17, n_estimators=150**.

**F1-мера** данной модели практически 0,57, **AUC-ROC** - 0,84, время обучения модели при этом - менее одной секунды.

Следующим этапом будем работать над борьбой с дисбалансом в данных. Будем улучшать метрики качества модели. По ходу борьбы метрики качества будем сравнивать с метриками модели **случайный лес**, полученными на данном этапе, так как данная модель в настоящий момент является лучшей.

## Борьба с дисбалансом

Для борьбы с дисбалансом классов испытаем три метода: **взвешенные классы, upsampling, downsampling**. Для последних двух методов сразу заготовим выборки.

In [26]:
# Подготовим увеличенные выборки (upsample)

oversample = SMOTE(random_state=RANDOM_STATE)
features_train_up, target_train_up = oversample.fit_resample(features_train, target_train)

sample_info(features_train_up)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,-0.134048,-0.078068,-0.369113,0.076163,0.816929,-1.550255,0.968496,0.331571,0.0,0.0,0.0
1,-1.010798,0.494555,-0.007415,0.136391,-0.896909,0.645055,0.968496,-0.727858,0.0,0.0,1.0
2,0.639554,1.353490,-1.454209,0.358435,-0.896909,0.645055,0.968496,-0.477006,1.0,0.0,1.0
3,-0.990168,2.116987,-1.092511,0.651725,-0.896909,0.645055,0.968496,-0.100232,0.0,0.0,0.0
4,0.567351,0.685430,0.715982,0.813110,0.816929,0.645055,0.968496,0.801922,0.0,0.0,0.0


(9562, 11)

In [27]:
# проверим веса классов

f'{Counter(target_train_up)}'

'Counter({0: 4781, 1: 4781})'

In [28]:
# Подготовим уменьшенные выборки (downsample)

undersample = RandomUnderSampler(random_state=RANDOM_STATE)
features_train_down, target_train_down = undersample.fit_resample(features_train, target_train)

sample_info(features_train_down)

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,1.485360,-0.841565,-1.092511,0.970141,0.816929,0.645055,-1.032529,-0.643853,0.0,0.0,1.0
1,-1.485274,-0.078068,-0.730812,1.436546,0.816929,-1.550255,0.968496,-1.344786,0.0,0.0,0.0
2,-1.712197,-1.414188,-1.454209,-1.232442,-0.896909,0.645055,-1.032529,-0.019356,0.0,1.0,1.0
3,-0.092789,-0.459816,-0.730812,0.101535,-0.896909,0.645055,0.968496,-1.366589,0.0,1.0,1.0
4,-1.402756,-0.364379,1.801078,0.437813,0.816929,-1.550255,-1.032529,0.343596,0.0,0.0,1.0


(2438, 11)

In [29]:
# проверим веса классов

f'{Counter(target_train_down)}'

'Counter({0: 1219, 1: 1219})'

In [30]:
# Обучим модель логистической регрессии, но с параметром class_weight='balanced'

model = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear', class_weight='balanced')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
f1_balanced = f1_score(target_valid, predicted_valid)

f'f1: {f1_balanced}, auc-roc: {calculation_of_aucroc(model, features_valid, target_valid)}'

'f1: 0.4750889679715302, auc-roc: 0.7725740281250446'

In [31]:
# поработаем с гиперпараметрами модели "Дерево решений" со взвешенными классами

best_depth = 0
best_f1 = f1_balanced
best_model = model
for depth in range(1, 20):
    model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth, class_weight='balanced')
    model_tree.fit(features_train, target_train)
    predicted_valid = model_tree.predict(features_valid)
    f1_tree = f1_score(target_valid, predicted_valid)
    if f1_tree > best_f1:
        best_depth = depth
        best_f1 = f1_tree
        best_model = model_tree

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}'

'f1: 0.5587044534412956, auc-roc: 0.8090671240258203, depth: 6'

In [32]:
# Теперь исследуем модель случайного леса

best_depth = 0
best_est = 0
best_f1 = best_f1
best_model = best_model

for depth in tqdm(range(1, 20)):
    for est in range(10, 170, 10):
        model = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=depth, n_estimators=est, class_weight='balanced')
        model.fit(features_train, target_train)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_depth = depth
            best_est = est
            best_f1 = f1
            best_model = model

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}, est: {best_est}'

100%|██████████| 19/19 [01:48<00:00,  5.73s/it]


'f1: 0.5974358974358975, auc-roc: 0.8529213074156081, depth: 10, est: 90'

In [33]:
%%time

# Измерим время обучения модели

best_model.fit(features_train, target_train)

Wall time: 507 ms


RandomForestClassifier(class_weight='balanced', max_depth=10, n_estimators=90,
                       random_state=12345)

На **взвешенных классах** лучшие результаты показа модель **случайного леса** с гиперпараметрами *max_depth=10, n_estimators=150*:
- *F1-мера*: 0.61
- *AUC-ROC*: 0.85
- время обучения модели - 627 мс

Дальше будем исследовать модели на увеличенных выборках.

In [34]:
# обучим модель логистической регрессии

model = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')
model.fit(features_train_up, target_train_up)
predicted_valid = model.predict(features_valid)
f1_up = f1_score(target_valid, predicted_valid)

f'f1: {f1_up}, auc-roc: {calculation_of_aucroc(model, features_valid, target_valid)}'

'f1: 0.4820788530465949, auc-roc: 0.7730683702129485'

In [35]:
# поработаем с гиперпараметрами модели "Дерево решений"

best_depth = 0
best_f1 = f1_up
best_model = model
for depth in range(1, 20):
    model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth)
    model_tree.fit(features_train_up, target_train_up)
    predicted_valid = model_tree.predict(features_valid)
    f1_tree = f1_score(target_valid, predicted_valid)
    if f1_tree > best_f1:
        best_depth = depth
        best_f1 = f1_tree
        best_model = model_tree

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}'

'f1: 0.5626423690205011, auc-roc: 0.8119743641505025, depth: 7'

In [36]:
# Теперь исследуем модель случайного леса

best_depth = 0
best_est = 0
best_f1 = best_f1
best_model = best_model

for depth in tqdm(range(1, 20)):
    for est in range(10, 170, 10):
        model = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=depth, n_estimators=est)
        model.fit(features_train_up, target_train_up)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_depth = depth
            best_est = est
            best_f1 = f1
            best_model = model

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}, est: {best_est}'

100%|██████████| 19/19 [02:56<00:00,  9.28s/it]


'f1: 0.6134969325153374, auc-roc: 0.8554883893190319, depth: 10, est: 130'

In [37]:
%%time

# Измерим время обучения модели

best_model.fit(features_train_up, target_train_up)

Wall time: 1.05 s


RandomForestClassifier(max_depth=10, n_estimators=130, random_state=12345)

С методом **upsampling** лучшие результаты показала модель **случайного леса** с гиперпараметрами *max_depth=12, n_estimators=40*:
- *F1-мера*: 0.61 (на 0,01 выше, чем при взвешенных классах)
- *AUC-ROC*: 0.85 (на 0,01 выше, чем при взвешенных классах)
- время обучения модели - 370 мс

Дальше будем исследовать модели на уменьшенных выборках.

In [38]:
# обучим модель логистической регрессии

model = LogisticRegression(random_state=RANDOM_STATE, solver='liblinear')
model.fit(features_train_down, target_train_down)
predicted_valid = model.predict(features_valid)
f1_down = f1_score(target_valid, predicted_valid)

f'f1: {f1_down}, auc-roc: {calculation_of_aucroc(model, features_valid, target_valid)}'

'f1: 0.47424511545293074, auc-roc: 0.7735563542032589'

In [39]:
# поработаем с гиперпараметрами модели "Дерево решений"

best_depth = 0
best_f1 = f1_down
best_model = model
for depth in range(1, 20):
    model_tree = DecisionTreeClassifier(random_state=RANDOM_STATE, max_depth=depth)
    model_tree.fit(features_train_down, target_train_down)
    predicted_valid = model_tree.predict(features_valid)
    f1_tree = f1_score(target_valid, predicted_valid)
    if f1_tree > best_f1:
        best_depth = depth
        best_f1 = f1_tree
        best_model = model_tree

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}'

'f1: 0.5444234404536863, auc-roc: 0.8120562246570204, depth: 6'

In [40]:
# Теперь исследуем модель случайного леса

best_depth = 0
best_est = 0
best_f1 = best_f1
best_model = best_model

for depth in tqdm(range(1, 20)):
    for est in range(10, 170, 10):
        model = RandomForestClassifier(random_state=RANDOM_STATE, max_depth=depth, n_estimators=est)
        model.fit(features_train_down, target_train_down)
        predicted_valid = model.predict(features_valid)
        f1 = f1_score(target_valid, predicted_valid)
        if f1 > best_f1:
            best_depth = depth
            best_est = est
            best_f1 = f1
            best_model = model

f'f1: {best_f1}, auc-roc: {calculation_of_aucroc(best_model, features_valid, target_valid)}, depth: {best_depth}, est: {best_est}'

100%|██████████| 19/19 [01:01<00:00,  3.21s/it]


'f1: 0.5752302968270215, auc-roc: 0.8467348784570169, depth: 7, est: 40'

In [41]:
%%time

# Измерим время обучения модели

best_model.fit(features_train_down, target_train_down)

Wall time: 88 ms


RandomForestClassifier(max_depth=7, n_estimators=40, random_state=12345)

<div class='alert alert-info'>
<b>Корректировка</b><br><br>
Проведено исследование различных моделей с учетом баланса классов и без учёта баланса:
<ul>
<li>Во всех исследованиях безусловным победителем оказалась модель RandomForestClassifier;</li>
<li>Самые высокие метрики качества оказались у модели обученной на сбалансированных методом *upsampling* данных и составили f1: 0.6084507042253521, auc-roc: 0.8505105074590915;</li>
<li>При этом следует отметить, что в данной модели всего 40 деревьев и время обучения оказалось одним из самых низких (среди других моделей "Случайного леса")</li>
</ul>
Гиперпараметры лучшей модели:
*RandomForestClassifier(max_depth=12, n_estimators=40, random_state=12345)*
Для дальнейшей проверки на тестовых данных, обучим модель с указанными гиперпараметрами на увеличенных выборках.
</div>

In [42]:
model = RandomForestClassifier(max_depth=10, n_estimators=130, random_state=RANDOM_STATE)
model.fit(features_train_up, target_train_up)

RandomForestClassifier(max_depth=10, n_estimators=130, random_state=12345)

## Тестирование модели

In [43]:
# Проверим модель на тестовой выборке

predicted_test = model.predict(features_test)
f1_test = f1_score(target_test, predicted_test)

f'f1-мера {f1_test}, auc-roc {calculation_of_aucroc(model, features_test, target_test)}'

'f1-мера 0.6376146788990825, auc-roc 0.8623046104417194'

In [44]:
# Сравним модель с константной

dummy_model = DummyClassifier(strategy='constant', constant=1)
dummy_model.fit(features_train, target_train)
f1_dummy = f1_score(dummy_model.predict(features_test), target_test)

In [45]:
# получим разницу метрик

f'{f1_test - f1_dummy}'

'0.28573993641865403'

In [46]:
# посчитаем метрику полноты

recall = recall_score(target_test, predicted_test)
recall

0.6510538641686182

**Общий вывод:**
- Среди всех проведенных исследований лучшей моделью была определена модель "Случайного леса" с максимальной глубиной 10 в ансамбле из 130 деревьев, обученная на сбалансированных методом upsampling тренировочных данных.
- При проверке данной модели на тестовой выборке она показала наилучшие метрики качества *F1-мера* и *AUC-ROC*, что говорит о правильном обучении модели.
- При сравнении с константной моделью, предсказывающей положительный класс, наша модель показывает метрику на 0,286 выше.
- Метрика recall, рассчитанная для тестовых данных, показывает, что 65 % положительных объектов модель смогла предсказать. Т.е. 65 % клиентов, желающих уйти из "Бета-Банка" модель определить смогла.

Подводя итог, можно сказать, что данную модель можно использовать для определения клиентов, желающих уйти из банка. Более 65 % будут определены.